In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

njobs = -1

In [2]:
train=pd.read_csv('../../1. Task and Data/train.csv',sep='|')
test=pd.read_csv('../../1. Task and Data/test.csv',sep='|')

In [290]:
print(train.fraud.value_counts())
print(train.fraud.value_counts() / len(train))

0    1775
1     104
Name: fraud, dtype: int64
0    0.944651
1    0.055349
Name: fraud, dtype: float64


# Feature Engineering

In [291]:
# for training set

train['scannedLineItems'] = train['scannedLineItemsPerSecond'] * train['totalScanTimeInSeconds']
train['pricePerScannedLineItem'] = train['grandTotal'] / train['scannedLineItems']
train['scansWithoutRegistrationPerScannedLineItem'] = train['scansWithoutRegistration'] / train['scannedLineItems']
train['quantityModificationsPerScannedLineItem'] = train['quantityModifications'] / train['scannedLineItems']
train['lineItemVoidsPerSecond'] = train['lineItemVoids'] / train['totalScanTimeInSeconds']
train['scansWithoutRegistrationPerSecond'] = train['scansWithoutRegistration'] / train['totalScanTimeInSeconds']
train['quantityModificationsPerSecond'] = train['quantityModifications'] / train['totalScanTimeInSeconds']
train['secondsPerEuro'] = train['totalScanTimeInSeconds'] / train['grandTotal']
train['lineItemVoidsPerEuro'] = train['lineItemVoids'] / train['grandTotal']
train['scansWithoutRegistrationPerEuro'] = train['scansWithoutRegistration'] / train['grandTotal']
train['quantityModificationsPerEuro'] = train['quantityModifications'] / train['grandTotal']


# for test set
test['scannedLineItems'] = test['scannedLineItemsPerSecond'] * test['totalScanTimeInSeconds']
test['pricePerScannedLineItem'] = test['grandTotal'] / test['scannedLineItems']
test['scansWithoutRegistrationPerScannedLineItem'] = test['scansWithoutRegistration'] / test['scannedLineItems']
test['quantityModificationsPerScannedLineItem'] = test['quantityModifications'] / test['scannedLineItems']
test['lineItemVoidsPerSecond'] = test['lineItemVoids'] / test['totalScanTimeInSeconds']
test['scansWithoutRegistrationPerSecond'] = test['scansWithoutRegistration'] / test['totalScanTimeInSeconds']
test['quantityModificationsPerSecond'] = test['quantityModifications'] / test['totalScanTimeInSeconds']
test['secondsPerEuro'] = test['totalScanTimeInSeconds'] / test['grandTotal']
test['lineItemVoidsPerEuro'] = test['lineItemVoids'] / test['grandTotal']
test['scansWithoutRegistrationPerEuro'] = test['scansWithoutRegistration'] / test['grandTotal']
test['quantityModificationsPerEuro'] = test['quantityModifications'] / test['grandTotal']

# Imputation for perEuro attributes

In [279]:
test.sort_values(by = "quantityModificationsPerEuro", ascending = False)['quantityModificationsPerEuro']

40562            inf
492399           inf
86833            inf
363211           inf
456994           inf
155622           inf
438266           inf
69401            inf
489766           inf
147774           inf
52962            inf
34895            inf
143886           inf
129034           inf
87373            inf
199015           inf
117536           inf
462632           inf
436235           inf
328053           inf
112020           inf
29990            inf
120191           inf
214052           inf
208504           inf
30187            inf
454691           inf
19792            inf
46561            inf
270427           inf
30423            inf
158772           inf
496219           inf
146252           inf
455867           inf
337680           inf
161945           inf
64223            inf
201272           inf
461791           inf
13695            inf
211326           inf
118032           inf
91248            inf
179013           inf
455538           inf
441018           inf
330870       

In [280]:
test.shape

(498121, 20)

In [292]:
test.loc[test['grandTotal'] == 0.00, 'secondsPerEuro'] = test['totalScanTimeInSeconds'] / 0.01
test.loc[test['grandTotal'] == 0.00, 'lineItemVoidsPerEuro'] = test['lineItemVoids'] / 0.01
test.loc[test['grandTotal'] == 0.00, 'scansWithoutRegistrationPerEuro'] = test['scansWithoutRegistration'] / 0.01
test.loc[test['grandTotal'] == 0.00, 'quantityModificationsPerEuro'] = test['quantityModifications'] / 0.01

In [283]:
test.sort_values(by = "quantityModificationsPerEuro", ascending = False)['quantityModificationsPerEuro']

31676     500.000000
112020    500.000000
287909    500.000000
214052    500.000000
455538    500.000000
46561     500.000000
53667     500.000000
208504    500.000000
52962     500.000000
330870    500.000000
270427    500.000000
133426    500.000000
497223    500.000000
461791    500.000000
143886    500.000000
389520    400.000000
328053    400.000000
147774    400.000000
3282      400.000000
161945    400.000000
91248     400.000000
371320    400.000000
118032    400.000000
176008    400.000000
283861    400.000000
34895     400.000000
43649     400.000000
337680    400.000000
19792     400.000000
299044    400.000000
13695     400.000000
102618    400.000000
236301    300.000000
156517    300.000000
462632    300.000000
211326    300.000000
129930    300.000000
436235    300.000000
428300    300.000000
256995    300.000000
129034    300.000000
496219    300.000000
58189     300.000000
104670    300.000000
492399    300.000000
199015    300.000000
16726     300.000000
117536    300

In [293]:
test.shape

(498121, 20)

# Larger training set -> Various Techniques

Easiest Way -> Apply rules found from Decision Trees

In [19]:
additional_no_frauds = test[test["trustLevel"] > 2.5]
additional_no_frauds = additional_no_frauds.assign(fraud = 0)

additional_frauds = test[(test["trustLevel"] < 1.5) &  (test["scannedLineItems"] > 20.5) & (test["valuePerSecond"] <= 0.118) & (test["scansWithoutRegistrationPerScannedLineItem"] > 0.168)]
additional_frauds = additional_frauds.assign(fraud = 1)


train = pd.concat([train, additional_no_frauds], ignore_index=True)
train = pd.concat([train, additional_frauds], ignore_index=True)

Generic Pseudo-Labeling -> current solution

In [294]:
from sklearn.base import BaseEstimator

class PseudoLabeler(BaseEstimator):
    '''
    Sci-kit learn wrapper for creating pseudo-lebeled estimators.
    '''
    
    def __init__(self, model, unlabled_data, features, target, sample_rate=0.2, seed=42):
        '''
        @sample_rate - percent of samples used as pseudo-labelled data
                       from the unlabled dataset
        '''
        assert sample_rate <= 1.0, 'Sample_rate should be between 0.0 and 1.0.'
        self.sample_rate = sample_rate
        self.seed = seed
        self.model = model
        self.model.seed = seed
        self.unlabled_data = unlabled_data
        self.features = features
        self.target = target

    def create_augmented_train(self, X, y):
        '''
        Create and return the augmented_train set that consists
        of pseudo-labeled and labeled data.
        '''        
        num_of_samples = int(len(self.unlabled_data) * self.sample_rate)
        
        # Train the model and creat the pseudo-labels
        self.model.fit(X[features], y)
        pseudo_labels = self.model.predict(self.unlabled_data[self.features])
        
        # Add the pseudo-labels to the test set
        pseudo_data = self.unlabled_data.copy(deep=True)
        pseudo_data[self.target] = pseudo_labels
        
        # Take a subset of the test set with pseudo-labels and append in onto
        # the training set
        sampled_pseudo_data = pseudo_data.sample(n=num_of_samples)
        temp_train = pd.concat([X, y], axis=1)
        augemented_train = pd.concat([sampled_pseudo_data, temp_train])

        return augemented_train

In [295]:
features = ['trustLevel', 'totalScanTimeInSeconds', 'lineItemVoids', 'scansWithoutRegistration',
            'scannedLineItemsPerSecond', 'valuePerSecond', 'lineItemVoidsPerPosition', 'scannedLineItems',
            'pricePerScannedLineItem', 'scansWithoutRegistrationPerScannedLineItem', 'quantityModificationsPerScannedLineItem',
            'lineItemVoidsPerSecond', 'quantityModificationsPerSecond']
target = 'fraud'

from sklearn.linear_model import LogisticRegression

# Best classifier of the training data set
PseudoLabelingModel = PseudoLabeler(
    model = LogisticRegression(fit_intercept = True, solver = 'newton-cg', max_iter = 10000),
    unlabled_data = test,
    features = features,
    target = target,
    sample_rate = 0.01
)

train = PseudoLabelingModel.create_augmented_train(train.drop('fraud',axis=1),train['fraud'])

In [296]:
train.shape

(6860, 21)

RandomOverSampler -> Simply Duplicate Fraud Instances from Train Data Set

In [ ]:
# !pip install imblearn  # might be necessary for installation
from imblearn.over_sampling import RandomOverSampler

def randomOverSampling(train):
    ros = RandomOverSampler(random_state=42)
    X_train_extended, Y_train_extended = ros.fit_resample(train.drop('fraud',axis=1), train['fraud'])

    new_train = pd.DataFrame(X_train_extended, columns=train.drop('fraud',axis=1).columns) 
    new_train["fraud"] = Y_train_extended
    return new_train
    
print(randomOverSampling(train).fraud.value_counts())
print(randomOverSampling(train).fraud.value_counts() / len(smoteOverSamling(train)))

train = randomOverSampling(train)

SMOTE -> Linear Combinations of already existing Fraud Instances from Train Data Set

In [2]:
#!pip install imblearn
from imblearn.over_sampling import SMOTE

def smoteOverSamling(train):
    sm = SMOTE(random_state=42, k_neighbors = 3, ratio = 0.5)
    X_train_extended, Y_train_extended = sm.fit_sample(train.drop('fraud',axis=1), train['fraud'])

    new_train = pd.DataFrame(X_train_extended, columns=train.drop('fraud',axis=1).columns) 
    new_train ["fraud"] = Y_train_extended
    return new_train
    
print(smoteOverSamling(train).fraud.value_counts())
print(smoteOverSamling(train).fraud.value_counts() / len(smoteOverSamling(train)))

train = smoteOverSamling(train)

NameError: name 'train' is not defined

ADASYN -> Linear Combinations plus additional random variance from Train Data Set

In [ ]:
from imblearn.over_sampling import ADASYN

def adasynOverSamling(train):
    ada = ADASYN(random_state=42, n_neighbors = 3, ratio = 0.5)
    X_train_extended, Y_train_extended = ada.fit_resample(train.drop('fraud',axis=1), train['fraud'])

    new_train = pd.DataFrame(X_train_extended, columns=train.drop('fraud',axis=1).columns) 
    new_train["fraud"] = Y_train_extended
    return new_train
    
print(adasynOverSamling(train).fraud.value_counts())
print(adasynOverSamling(train).fraud.value_counts() / len(train))

train = adasynOverSamling(train)

# Resulting fraud distribution of the new data set

In [297]:
print(train.fraud.value_counts())
print(train.fraud.value_counts() / len(train))

0    6512
1     348
Name: fraud, dtype: int64
0    0.949271
1    0.050729
Name: fraud, dtype: float64


In [298]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer


def my_custom_loss_func(y_true, y_pred):
    cm = confusion_matrix(y_true,y_pred)
    tn, fp, fn, tp = cm.ravel()
    score = ((-25)*fp + (-5)*fn + 5*tp) / len(y_true)
    return (score)

my_custom_score = make_scorer(my_custom_loss_func, greater_is_better=True)

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut

skf = StratifiedKFold(n_splits=10)

In [302]:
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm.classes import OneClassSVM
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
from sklearn.neighbors.classification import RadiusNeighborsClassifier
from sklearn.neighbors.classification import KNeighborsClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model.stochastic_gradient import SGDClassifier
from sklearn.linear_model.ridge import RidgeClassifierCV
from sklearn.linear_model.ridge import RidgeClassifier
from sklearn.linear_model.passive_aggressive import PassiveAggressiveClassifier
from sklearn.gaussian_process.gpc import GaussianProcessClassifier
from sklearn.ensemble.voting_classifier import VotingClassifier
from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.bagging import BaggingClassifier
from sklearn.ensemble.forest import ExtraTreesClassifier
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
# from sklearn.mixture import DPGMM
#from sklearn.mixture import GMM
#from sklearn.mixture import GaussianMixture
#from sklearn.mixture import VBGMM

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


model_tuning_factory = [
    GridSearchCV(LogisticRegression(max_iter = 10000), 
                 dict(# penalty = ['l1','l2'],  # automatic regularization  -> option 'l1' doesnt work with all solvers and leads to errors
                     solver = ['newton-cg', 'lbfgs', 'liblinear'], 
                     fit_intercept = [True, False]),
                    #  solver = ['lbfgs', 'liblinear', 'sag', 'saga']),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(KNeighborsClassifier(), 
                 dict(n_neighbors = range(1,4),
                      weights = ['uniform', 'distance']),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(NearestCentroid(),
                 dict(),
                 cv = skf,
                 scoring = my_custom_score), 
    GridSearchCV(DecisionTreeClassifier(),
                 dict(criterion = ['entropy', 'gini']),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(ExtraTreeClassifier(),
                 dict(criterion = ['entropy', 'gini']),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(ExtraTreesClassifier(),
                 dict(criterion = ['entropy', 'gini']),
                      # n_estimators  = range(1,150)),
                 cv = skf,
                 scoring = my_custom_score),    
    GridSearchCV(RandomForestClassifier(),
                 dict(criterion = ['entropy', 'gini']),
                     # n_estimators  = range(1,150)),
                 cv = skf,
                 scoring = my_custom_score),    
    GridSearchCV(BernoulliNB(),
                 dict(binarize  = np.arange(0.0, 2.0, 0.1)),
                 cv = skf,
                 scoring = my_custom_score), 
    GridSearchCV(GaussianNB(),
                 dict(),
                 cv = skf,
                 scoring = my_custom_score), 
    GridSearchCV(AdaBoostClassifier(),
                 dict(),
                 #dict(n_estimators = range(1,150)),
                 cv = skf,
                 scoring = my_custom_score), 
    GridSearchCV(GradientBoostingClassifier(),
                 dict(),
                 #dict(n_estimators = range(1,150)),
                 #     learning_rate = np.arange(0.01, 1.0, 0.01)),
                 cv = skf,
                 scoring = my_custom_score),                     
    GridSearchCV(BaggingClassifier(), 
                 dict(),
                 #dict(n_estimators = range(1,150)),
                 cv = skf,
                 scoring = my_custom_score), 
    GridSearchCV(Perceptron(),
                 dict(),
                 cv = skf,
                 scoring = my_custom_score), 
    GridSearchCV(LinearDiscriminantAnalysis(),
                 dict(),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(QuadraticDiscriminantAnalysis(),
                 dict(),
                 cv = skf,
                 scoring = my_custom_score),
 #   GridSearchCV(SVC(), # takes too much time
 #                dict(),
 #                cv = skf,
 #                scoring = my_custom_score)
]           
                 
                 
iterations = 10                 
               
model_tuning_factory_randomized = [
    RandomizedSearchCV(LogisticRegression(max_iter = 10000), 
                 dict(# penalty = ['l1','l2'],  # automatic regularization  -> option 'l1' doesnt work with all solvers and leads to errors
                       solver = ['newton-cg', 'lbfgs', 'liblinear'],
                       fit_intercept = [True, False]),
                    #  solver = ['lbfgs', 'liblinear', 'sag', 'saga']),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations),
    RandomizedSearchCV(KNeighborsClassifier(), 
                 dict(n_neighbors = range(1,4),
                      weights = ['uniform', 'distance']),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations),
    RandomizedSearchCV(NearestCentroid(),
                 dict(),
                 cv = skf,
                 scoring = my_custom_score), 
    RandomizedSearchCV(DecisionTreeClassifier(),
                 dict(criterion = ['entropy', 'gini']),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations),
    RandomizedSearchCV(ExtraTreeClassifier(),
                 dict(criterion = ['entropy', 'gini']),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations),
    RandomizedSearchCV(ExtraTreesClassifier(),
                 dict(criterion = ['entropy', 'gini'],
                      n_estimators  = range(1,150)),
                 cv = skf,
                 scoring = my_custom_score),    
    RandomizedSearchCV(RandomForestClassifier(),
                 dict(criterion = ['entropy', 'gini'],
                      n_estimators  = range(1,150)),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations),    
    RandomizedSearchCV(BernoulliNB(),
                 dict(binarize  = np.arange(0.0, 1.0, 0.1)),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations), 
    RandomizedSearchCV(GaussianNB(),
                 dict(),
                 cv = skf,
                 scoring = my_custom_score), 
    RandomizedSearchCV(AdaBoostClassifier(),
                 dict(n_estimators = range(1,150)),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations),
    RandomizedSearchCV(GradientBoostingClassifier(),
                 dict(n_estimators = range(1,150),
                      learning_rate = np.arange(0.01, 1.0, 0.01)),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations),                    
    RandomizedSearchCV(BaggingClassifier(), 
                 dict(n_estimators = range(1,150)),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations),
    RandomizedSearchCV(Perceptron(),
                 dict(),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations),
    RandomizedSearchCV(LinearDiscriminantAnalysis(),
                 dict(),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations),
    RandomizedSearchCV(QuadraticDiscriminantAnalysis(),
                 dict(),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations),
    RandomizedSearchCV(SVC(),
                 dict(),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = iterations)
]

In [300]:
Y = train['fraud']
X = train.drop('fraud',axis=1)

# Take a smaller sample for quicker training -> needs to be adjusted together with the amount of pseudo labelled instances

In [33]:
# train = train.sample(50000)

In [271]:
# print(train.fraud.value_counts())
# print(train.fraud.value_counts() / len(train))

0    49283
1     2408
Name: fraud, dtype: int64
0    0.953415
1    0.046585
Name: fraud, dtype: float64


In [303]:
import time
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

result_table = pd.DataFrame(columns=["Model", "Data Preparation", "Feature Count", "Features", "Optimal Parameters", "Monetary Value Per Instance - Mean", "Monetary Value Per Instance - Standard Deviation", "Time needed", "Raw Model"])


# three types of data preparation: No preparation, MaxMinScaler, StandardScaler
for data_preparation_step in range(1,4):
    if (data_preparation_step == 1):  
        X_scaled = X
        data_preparation = "No Scaling"
    elif (data_preparation_step == 2):
        feature_scaler = MinMaxScaler()  
        data_preparation = "MinMaxScaler"
    elif (data_preparation_step == 3):
        feature_scaler = StandardScaler()  
        X_scaled = pd.DataFrame(feature_scaler.fit_transform(X.values), columns=X.columns, index=X.index)
        data_preparation = "StandardScaler"
    elif (data_preparation_step == 4):
        transformer = FunctionTransformer(np.log1p, validate=True)
        X_scaled = pd.DataFrame(transformer.transform(X), columns=X.columns, index=X.index)
        data_preparation = "LogScaler"    
    

    for feature_count in range(1,len(list(X))+1):

        for model in model_tuning_factory:   # replace with model_tuning_factory_randomized for faster results
            
            start_time = time.time()
            
            best_features = SelectKBest(f_classif, k=feature_count).fit(X_scaled,Y)
            best_feature_list = X.columns[best_features.get_support()]
            X_selected_features = X[best_feature_list]
            
            model.seed = 42
            model.fit(X_selected_features,Y)  
            model_name = model.best_estimator_.__class__.__name__
            score_mean = model.cv_results_['mean_test_score'][model.best_index_]
            score_std = model.cv_results_['std_test_score'][model.best_index_]
            
            end_time = time.time()
            
            print("Finished " + model.best_estimator_.__class__.__name__ + " with " + data_preparation + " and " + str(feature_count) + " features after " + str(end_time - start_time) + " seconds")

            result_table = result_table.append({
             "Model": model_name,
             "Data Preparation": data_preparation,
             "Feature Count": feature_count,
             "Features": best_feature_list.values,
             "Optimal Parameters": model.best_params_,
             "Monetary Value Per Instance - Mean":  score_mean,
             "Monetary Value Per Instance - Standard Deviation": score_std,
             "Time needed": end_time - start_time,    
             "Raw Model": model.best_estimator_
              }, ignore_index=True)
    
result_table.sort_values(by = "Monetary Value Per Instance - Mean", ascending = False)

Finished LogisticRegression with No Scaling and 1 features after 0.563493013381958 seconds
Finished KNeighborsClassifier with No Scaling and 1 features after 5.618963956832886 seconds
Finished NearestCentroid with No Scaling and 1 features after 0.06480073928833008 seconds
Finished DecisionTreeClassifier with No Scaling and 1 features after 0.12366914749145508 seconds
Finished ExtraTreeClassifier with No Scaling and 1 features after 0.15960240364074707 seconds
Finished ExtraTreesClassifier with No Scaling and 1 features after 0.3291196823120117 seconds
Finished RandomForestClassifier with No Scaling and 1 features after 0.34707021713256836 seconds
Finished BernoulliNB with No Scaling and 1 features after 1.6076765060424805 seconds
Finished GaussianNB with No Scaling and 1 features after 0.07579588890075684 seconds
Finished AdaBoostClassifier with No Scaling and 1 features after 1.015735387802124 seconds
Finished GradientBoostingClassifier with No Scaling and 1 features after 0.91555094

C:\Users\ahuem\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ahuem\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Finished LogisticRegression with No Scaling and 3 features after 4.589951276779175 seconds
Finished KNeighborsClassifier with No Scaling and 3 features after 3.888800859451294 seconds
Finished NearestCentroid with No Scaling and 3 features after 0.06961750984191895 seconds
Finished DecisionTreeClassifier with No Scaling and 3 features after 0.18504595756530762 seconds
Finished ExtraTreeClassifier with No Scaling and 3 features after 0.13962626457214355 seconds
Finished ExtraTreesClassifier with No Scaling and 3 features after 0.4297170639038086 seconds
Finished RandomForestClassifier with No Scaling and 3 features after 0.5043587684631348 seconds
Finished BernoulliNB with No Scaling and 3 features after 1.3127939701080322 seconds
Finished GaussianNB with No Scaling and 3 features after 0.0703439712524414 seconds
Finished AdaBoostClassifier with No Scaling and 3 features after 1.2548248767852783 seconds
Finished GradientBoostingClassifier with No Scaling and 3 features after 1.473753929

C:\Users\ahuem\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ahuem\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
C:\Users\ahuem\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:422: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
C:\Users\ahuem\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ahuem\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Finished LogisticRegression with No Scaling and 4 features after 4.415508031845093 seconds
Finished KNeighborsClassifier with No Scaling and 4 features after 4.105004072189331 seconds
Finished NearestCentroid with No Scaling and 4 features after 0.0688176155090332 seconds
Finished DecisionTreeClassifier with No Scaling and 4 features after 0.218519926071167 seconds
Finished ExtraTreeClassifier with No Scaling and 4 features after 0.14079713821411133 seconds
Finished ExtraTreesClassifier with No Scaling and 4 features after 0.44333910942077637 seconds
Finished RandomForestClassifier with No Scaling and 4 features after 0.636293888092041 seconds
Finished BernoulliNB with No Scaling and 4 features after 1.3940742015838623 seconds
Finished GaussianNB with No Scaling and 4 features after 0.07629013061523438 seconds
Finished AdaBoostClassifier with No Scaling and 4 features after 1.5189731121063232 seconds
Finished GradientBoostingClassifier with No Scaling and 4 features after 1.77829241752

...

Finished GradientBoostingClassifier with StandardScaler and 9 features after 2.662745237350464 seconds
Finished BaggingClassifier with StandardScaler and 9 features after 0.6637532711029053 seconds
Finished Perceptron with StandardScaler and 9 features after 0.09170961380004883 seconds
Finished LinearDiscriminantAnalysis with StandardScaler and 9 features after 0.1252269744873047 seconds
Finished QuadraticDiscriminantAnalysis with StandardScaler and 9 features after 0.09175467491149902 seconds
Finished LogisticRegression with StandardScaler and 10 features after 10.348755359649658 seconds
Finished KNeighborsClassifier with StandardScaler and 10 features after 4.809107065200806 seconds
Finished NearestCentroid with StandardScaler and 10 features after 0.09372615814208984 seconds
Finished DecisionTreeClassifier with StandardScaler and 10 features after 0.3760683536529541 seconds
Finished ExtraTreeClassifier with StandardScaler and 10 features after 0.14629507064819336 seconds
Finished Ex

Finished ExtraTreesClassifier with StandardScaler and 15 features after 0.5529699325561523 seconds
Finished RandomForestClassifier with StandardScaler and 15 features after 0.9576010704040527 seconds
Finished BernoulliNB with StandardScaler and 15 features after 2.0389018058776855 seconds
Finished GaussianNB with StandardScaler and 15 features after 0.15588784217834473 seconds
Finished AdaBoostClassifier with StandardScaler and 15 features after 3.0646209716796875 seconds
Finished GradientBoostingClassifier with StandardScaler and 15 features after 4.278324604034424 seconds
Finished BaggingClassifier with StandardScaler and 15 features after 1.2160918712615967 seconds
Finished Perceptron with StandardScaler and 15 features after 0.1385498046875 seconds
Finished LinearDiscriminantAnalysis with StandardScaler and 15 features after 0.19586706161499023 seconds
Finished QuadraticDiscriminantAnalysis with StandardScaler and 15 features after 0.11866116523742676 seconds


C:\Users\ahuem\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\ahuem\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Finished LogisticRegression with StandardScaler and 16 features after 20.700650691986084 seconds
Finished KNeighborsClassifier with StandardScaler and 16 features after 5.777050495147705 seconds
Finished NearestCentroid with StandardScaler and 16 features after 0.07782149314880371 seconds
Finished DecisionTreeClassifier with StandardScaler and 16 features after 0.6095733642578125 seconds
Finished ExtraTreeClassifier with StandardScaler and 16 features after 0.15678048133850098 seconds
Finished ExtraTreesClassifier with StandardScaler and 16 features after 0.5718472003936768 seconds
Finished RandomForestClassifier with StandardScaler and 16 features after 1.1170318126678467 seconds
Finished BernoulliNB with StandardScaler and 16 features after 2.2417848110198975 seconds
Finished GaussianNB with StandardScaler and 16 features after 0.10673761367797852 seconds
Finished AdaBoostClassifier with StandardScaler and 16 features after 3.36482310295105 seconds
Finished GradientBoostingClassifier

,Model,Data Preparation,Feature Count,Features,Optimal Parameters,Monetary Value Per Instance - Mean,Monetary Value Per Instance - Standard Deviation,Time needed,Raw Model
450,LogisticRegression,MinMaxScaler,11,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.551737,"LogisticRegression(C=1.0, class_weight=None, d..."
765,LogisticRegression,StandardScaler,12,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.944594,"LogisticRegression(C=1.0, class_weight=None, d..."
435,LogisticRegression,MinMaxScaler,10,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,9.837662,"LogisticRegression(C=1.0, class_weight=None, d..."
735,LogisticRegression,StandardScaler,10,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.348755,"LogisticRegression(C=1.0, class_weight=None, d..."
135,LogisticRegression,No Scaling,10,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.536294,"LogisticRegression(C=1.0, class_weight=None, d..."
465,LogisticRegression,MinMaxScaler,12,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.011464,"LogisticRegression(C=1.0, class_weight=None, d..."
150,LogisticRegression,No Scaling,11,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.962854,"LogisticRegression(C=1.0, class_weight=None, d..."
750,LogisticRegression,StandardScaler,11,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.222572,"LogisticRegression(C=1.0, class_weight=None, d..."
165,LogisticRegression,No Scaling,12,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.201692,"LogisticRegression(C=1.0, class_weight=None, d..."
720,LogisticRegression,StandardScaler,9,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.212099,0.050064,9.673945,"LogisticRegression(C=1.0, class_weight=None, d..."


# Restore the saved result table to avoid recomputation

In [3]:
#result_table.to_pickle("result_table_extended_training_set.pkl")
result_table = pd.read_pickle("result_table_extended_training_set.pkl")

In [306]:
result_table = result_table.sort_values(by = "Monetary Value Per Instance - Mean", ascending = False)

result_table.to_excel("Result-Extended Train Set.xlsx")

result_table

,Model,Data Preparation,Feature Count,Features,Optimal Parameters,Monetary Value Per Instance - Mean,Monetary Value Per Instance - Standard Deviation,Time needed,Raw Model
450,LogisticRegression,MinMaxScaler,11,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.551737,"LogisticRegression(C=1.0, class_weight=None, d..."
465,LogisticRegression,MinMaxScaler,12,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.011464,"LogisticRegression(C=1.0, class_weight=None, d..."
165,LogisticRegression,No Scaling,12,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.201692,"LogisticRegression(C=1.0, class_weight=None, d..."
750,LogisticRegression,StandardScaler,11,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.222572,"LogisticRegression(C=1.0, class_weight=None, d..."
150,LogisticRegression,No Scaling,11,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.962854,"LogisticRegression(C=1.0, class_weight=None, d..."
765,LogisticRegression,StandardScaler,12,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.944594,"LogisticRegression(C=1.0, class_weight=None, d..."
135,LogisticRegression,No Scaling,10,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.536294,"LogisticRegression(C=1.0, class_weight=None, d..."
435,LogisticRegression,MinMaxScaler,10,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,9.837662,"LogisticRegression(C=1.0, class_weight=None, d..."
735,LogisticRegression,StandardScaler,10,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.348755,"LogisticRegression(C=1.0, class_weight=None, d..."
720,LogisticRegression,StandardScaler,9,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.212099,0.050064,9.673945,"LogisticRegression(C=1.0, class_weight=None, d..."


In [307]:
result_table_aggregated = pd.DataFrame(columns=["Model", "Data Preparation", "Feature Count", "Features", "Optimal Parameters", "Monetary Value Per Instance - Mean", "Monetary Value Per Instance - Standard Deviation", "Time needed", "Raw Model"])


for model in list(set(result_table["Model"].values)):
    sub_table = result_table[result_table["Model"] == model]
    result_table_aggregated = result_table_aggregated.append({
        "Model": model,
        "Data Preparation": sub_table.loc[sub_table["Monetary Value Per Instance - Mean"].argmax()]["Data Preparation"],
        "Feature Count": sub_table.loc[sub_table["Monetary Value Per Instance - Mean"].argmax()]["Feature Count"],
        "Features": sub_table.loc[sub_table["Monetary Value Per Instance - Mean"].argmax()]["Features"],
        "Optimal Parameters": sub_table.loc[sub_table["Monetary Value Per Instance - Mean"].argmax()]["Optimal Parameters"],
        "Monetary Value Per Instance - Mean": sub_table.loc[sub_table["Monetary Value Per Instance - Mean"].argmax()]["Monetary Value Per Instance - Mean"],
        "Monetary Value Per Instance - Standard Deviation": sub_table.loc[sub_table["Monetary Value Per Instance - Mean"].argmax()]["Monetary Value Per Instance - Standard Deviation"],
        "Time needed":  sub_table.loc[sub_table["Monetary Value Per Instance - Mean"].argmax()]["Time needed"],
        "Raw Model": sub_table.loc[sub_table["Monetary Value Per Instance - Mean"].argmax()]["Raw Model"]
        }, ignore_index=True)

result_table_aggregated.sort_values(by = "Monetary Value Per Instance - Mean", ascending = False).to_excel("Result-Extended Train Set-Aggregated.xlsx")

result_table_aggregated.sort_values(by = "Monetary Value Per Instance - Mean", ascending = False)

,Model,Data Preparation,Feature Count,Features,Optimal Parameters,Monetary Value Per Instance - Mean,Monetary Value Per Instance - Standard Deviation,Time needed,Raw Model
5,LogisticRegression,MinMaxScaler,11,"[trustLevel, totalScanTimeInSeconds, lineItemV...","{'fit_intercept': True, 'solver': 'newton-cg'}",0.214286,0.051290,10.551737,"LogisticRegression(C=1.0, class_weight=None, d..."
12,AdaBoostClassifier,StandardScaler,9,"[trustLevel, totalScanTimeInSeconds, lineItemV...",{},0.145044,0.082631,1.984250,"(DecisionTreeClassifier(class_weight=None, cri..."
1,GradientBoostingClassifier,StandardScaler,15,"[trustLevel, totalScanTimeInSeconds, lineItemV...",{},0.134840,0.063635,4.278325,([DecisionTreeRegressor(criterion='friedman_ms...
2,ExtraTreesClassifier,StandardScaler,13,"[trustLevel, totalScanTimeInSeconds, lineItemV...",{'criterion': 'gini'},0.092566,0.055056,0.526340,"(ExtraTreeClassifier(class_weight=None, criter..."
4,RandomForestClassifier,MinMaxScaler,10,"[trustLevel, totalScanTimeInSeconds, lineItemV...",{'criterion': 'entropy'},0.090379,0.084589,0.804199,"(DecisionTreeClassifier(class_weight=None, cri..."
13,BaggingClassifier,No Scaling,8,"[trustLevel, totalScanTimeInSeconds, scansWith...",{},0.079446,0.075695,0.629416,"(DecisionTreeClassifier(class_weight=None, cri..."
7,DecisionTreeClassifier,StandardScaler,10,"[trustLevel, totalScanTimeInSeconds, lineItemV...",{'criterion': 'entropy'},-0.023324,0.121778,0.376068,"DecisionTreeClassifier(class_weight=None, crit..."
8,LinearDiscriminantAnalysis,MinMaxScaler,19,"[trustLevel, totalScanTimeInSeconds, grandTota...",{},-0.077259,0.045458,0.203597,"LinearDiscriminantAnalysis(n_components=None, ..."
9,GaussianNB,StandardScaler,3,"[trustLevel, totalScanTimeInSeconds, scannedLi...",{},-0.099125,0.091143,0.071010,"GaussianNB(priors=None, var_smoothing=1e-09)"
10,QuadraticDiscriminantAnalysis,No Scaling,3,"[trustLevel, totalScanTimeInSeconds, scannedLi...",{},-0.106414,0.081819,0.094721,"QuadraticDiscriminantAnalysis(priors=None, reg..."


In [308]:
best_model = result_table_aggregated.loc[result_table_aggregated["Monetary Value Per Instance - Mean"].argmax()]["Raw Model"]
best_model_features = result_table_aggregated.loc[result_table_aggregated["Monetary Value Per Instance - Mean"].argmax()]["Features"]

print(best_model)
print(best_model_features)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)
['trustLevel' 'totalScanTimeInSeconds' 'lineItemVoids'
 'scansWithoutRegistration' 'valuePerSecond' 'lineItemVoidsPerPosition'
 'scannedLineItems' 'pricePerScannedLineItem'
 'scansWithoutRegistrationPerScannedLineItem'
 'quantityModificationsPerScannedLineItem'
 'quantityModificationsPerSecond']


In [309]:
from sklearn.metrics import confusion_matrix

def get_monetary_value(cm):
        tn, fp, fn, tp = cm.ravel()
        print("True negative: ", tn)
        print("False positive: ", fp)
        print("False negative: ", fn)
        print("True positive: ", tp)
        score = (-25)*fp + (-5)*fn + 5*tp
        print(str(score) + " for " + str(sum(sum(cm))) + " instances in the test set")
        print(str(score/sum(sum(cm))) + " per instance in the test set")
        return score
    
cm = confusion_matrix(Y , best_model.predict(X[best_model_features]))

monetary_value = get_monetary_value(cm)

True negative:  6504
False positive:  8
False negative:  5
True positive:  343
1490 for  6860  instances in the test set
0.21720116618075802  per instance in the test set
